In [ ]:
! wget https://github.com/erkansirin78/datasets/tree/master/retail_db

In [1]:
import findspark
findspark.init("/opt/spark")

In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *

In [3]:
spark = SparkSession \
    .builder \
    .appName("retailorder") \
    .config("spark.jars", "/opt/spark/jars/postgresql-42.3.5.jar") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog").getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e411c122-bdea-417c-9459-2a8b1623584b;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 1072ms :: artifacts dl 27ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0

In [4]:
df_orders = spark.read.option("header", True) \
    .option("inferSchema", True) \
    .csv("orders.csv")

df_order_items = spark.read.option("header", True) \
    .option("inferSchema", True) \
    .csv("order_items.csv")

df_products = spark.read.option("header", True) \
    .option("inferSchema", True) \
    .csv("products.csv")

df_categories = spark.read.option("header", True) \
    .option("inferSchema", True) \
    .csv("categories.csv")

df_departments = spark.read.option("header", True) \
    .option("inferSchema", True) \
    .csv("departments.csv")

df_customers = spark.read.option("header", True) \
    .option("inferSchema", True) \
    .csv("customers.csv")

In [5]:
df_orders.show(5)

df_order_items.show(5)

+-------+-------------------+---------------+---------------+
|orderId|          orderDate|orderCustomerId|    orderStatus|
+-------+-------------------+---------------+---------------+
|      1|2013-07-25 00:00:00|          11599|         CLOSED|
|      2|2013-07-25 00:00:00|            256|PENDING_PAYMENT|
|      3|2013-07-25 00:00:00|          12111|       COMPLETE|
|      4|2013-07-25 00:00:00|           8827|         CLOSED|
|      5|2013-07-25 00:00:00|          11318|       COMPLETE|
+-------+-------------------+---------------+---------------+
only showing top 5 rows

+-------------+----------------+------------------+-----------------+-----------------+---------------------+
|orderItemName|orderItemOrderId|orderItemProductId|orderItemQuantity|orderItemSubTotal|orderItemProductPrice|
+-------------+----------------+------------------+-----------------+-----------------+---------------------+
|            1|               1|               957|                1|           299.98|

## 1. task

In [6]:
t1 = df_order_items.select("orderItemOrderId").distinct().count()

In [7]:
t1

57431

## 2. Task

In [8]:
t2_1 = df_orders.count()
t2_1

68883

In [9]:
t2_2 = df_order_items.count()
t2_2

172198

## 3. Task

In [10]:
t3_joined = df_order_items.join(df_orders, df_order_items["orderItemOrderId"] == df_orders["orderId"], "inner")

In [11]:
t3_joined.limit(5).toPandas()

,orderItemName,orderItemOrderId,orderItemProductId,orderItemQuantity,orderItemSubTotal,orderItemProductPrice,orderId,orderDate,orderCustomerId,orderStatus
0,1,1,957,1,299.98,299.98,1,2013-07-25,11599,CLOSED
1,2,2,1073,1,199.99,199.99,2,2013-07-25,256,PENDING_PAYMENT
2,3,2,502,5,250.00,50.00,2,2013-07-25,256,PENDING_PAYMENT
3,4,2,403,1,129.99,129.99,2,2013-07-25,256,PENDING_PAYMENT
4,5,4,897,2,49.98,24.99,4,2013-07-25,8827,CLOSED


In [12]:
df_products.show(5)

+---------+-----------------+--------------------+------------------+------------+--------------------+
|productId|productCategoryId|         productName|productDescription|productPrice|        productImage|
+---------+-----------------+--------------------+------------------+------------+--------------------+
|        1|                2|Quest Q64 10 FT. ...|              null|       59.98|http://images.acm...|
|        2|                2|Under Armour Men'...|              null|      129.99|http://images.acm...|
|        3|                2|Under Armour Men'...|              null|       89.99|http://images.acm...|
|        4|                2|Under Armour Men'...|              null|       89.99|http://images.acm...|
|        5|                2|Riddell Youth Rev...|              null|      199.99|http://images.acm...|
+---------+-----------------+--------------------+------------------+------------+--------------------+
only showing top 5 rows



In [13]:
t3_joined_product = t3_joined \
.join(df_products, t3_joined["orderItemOrderId"] == df_products["productId"], "inner")

In [14]:
t3_joined_product.limit(5).toPandas()

,orderItemName,orderItemOrderId,orderItemProductId,orderItemQuantity,orderItemSubTotal,orderItemProductPrice,orderId,orderDate,orderCustomerId,orderStatus,productId,productCategoryId,productName,productDescription,productPrice,productImage
0,1,1,957,1,299.98,299.98,1,2013-07-25,11599,CLOSED,1,2,Quest Q64 10 FT. x 10 FT. Slant Leg Instant U,None,59.98,http://images.acmesports.sports/Quest+Q64+10+F...
1,2,2,1073,1,199.99,199.99,2,2013-07-25,256,PENDING_PAYMENT,2,2,Under Armour Men's Highlight MC Football Clea,None,129.99,http://images.acmesports.sports/Under+Armour+M...
2,3,2,502,5,250.00,50.00,2,2013-07-25,256,PENDING_PAYMENT,2,2,Under Armour Men's Highlight MC Football Clea,None,129.99,http://images.acmesports.sports/Under+Armour+M...
3,4,2,403,1,129.99,129.99,2,2013-07-25,256,PENDING_PAYMENT,2,2,Under Armour Men's Highlight MC Football Clea,None,129.99,http://images.acmesports.sports/Under+Armour+M...
4,5,4,897,2,49.98,24.99,4,2013-07-25,8827,CLOSED,4,2,Under Armour Men's Renegade D Mid Football Cl,None,89.99,http://images.acmesports.sports/Under+Armour+M...


In [15]:
t3_joined_canceled = t3_joined_product.filter(t3_joined_product["orderStatus"] == "CANCELED")

product_sales = t3_joined_canceled.groupBy("productName").agg(sum("orderItemSubTotal").alias("TotalSales"))

product_sales = product_sales.orderBy("TotalSales", ascending=False)

product_sales.limit(5).toPandas()

,productName,TotalSales
0,Ogio Sport Golf Shoes,1129.75
1,Quest Q64 10 FT. x 10 FT. Slant Leg Instant U,1079.89
2,Nike Women's Pro Hyperwarm Half Zip Long Slee,979.88
3,Nike Men's KD VI Basketball Shoe,969.90
4,Garmin Approach S3 Golf GPS Watch,819.84


In [17]:
product_sales.write.parquet("product_sales.parquet")

In [18]:
product_sales_parquet = spark.read.parquet("product_sales.parquet")
product_sales_parquet.limit(5).toPandas()

,productName,TotalSales
0,Ogio Sport Golf Shoes,1129.75
1,Quest Q64 10 FT. x 10 FT. Slant Leg Instant U,1079.89
2,Nike Women's Pro Hyperwarm Half Zip Long Slee,979.88
3,Nike Men's KD VI Basketball Shoe,969.90
4,Garmin Approach S3 Golf GPS Watch,819.84


## 4. Task

In [20]:
df_categories.limit(5).toPandas()

,categoryId,categoryDepartmentId,categoryName
0,1,2,Football
1,2,2,Soccer
2,3,2,Baseball & Softball
3,4,2,Basketball
4,5,2,Lacrosse


In [22]:
t3_joined_canceled.limit(5).toPandas()

,orderItemName,orderItemOrderId,orderItemProductId,orderItemQuantity,orderItemSubTotal,orderItemProductPrice,orderId,orderDate,orderCustomerId,orderStatus,productId,productCategoryId,productName,productDescription,productPrice,productImage
0,124,50,403,1,129.99,129.99,50,2013-07-25,5225,CANCELED,50,4,Quest Q64 10 FT. x 10 FT. Slant Leg Instant U,None,59.98,http://images.acmesports.sports/Quest+Q64+10+F...
1,125,50,957,1,299.98,299.98,50,2013-07-25,5225,CANCELED,50,4,Quest Q64 10 FT. x 10 FT. Slant Leg Instant U,None,59.98,http://images.acmesports.sports/Quest+Q64+10+F...
2,254,112,1014,2,99.96,49.98,112,2013-07-26,5375,CANCELED,112,6,Nike Women's Pro Hyperwarm Half Zip Long Slee,None,31.97,http://images.acmesports.sports/Nike+Women%27s...
3,255,112,642,4,120.00,30.00,112,2013-07-26,5375,CANCELED,112,6,Nike Women's Pro Hyperwarm Half Zip Long Slee,None,31.97,http://images.acmesports.sports/Nike+Women%27s...
4,256,112,365,2,119.98,59.99,112,2013-07-26,5375,CANCELED,112,6,Nike Women's Pro Hyperwarm Half Zip Long Slee,None,31.97,http://images.acmesports.sports/Nike+Women%27s...


In [23]:
t4_joined_product_category = t3_joined_canceled \
.join(df_categories, t3_joined_canceled["productCategoryId"] == df_categories["categoryId"], "inner")

In [24]:
t4_joined_product_category.limit(5).toPandas()

,orderItemName,orderItemOrderId,orderItemProductId,orderItemQuantity,orderItemSubTotal,orderItemProductPrice,orderId,orderDate,orderCustomerId,orderStatus,productId,productCategoryId,productName,productDescription,productPrice,productImage,categoryId,categoryDepartmentId,categoryName
0,124,50,403,1,129.99,129.99,50,2013-07-25,5225,CANCELED,50,4,Quest Q64 10 FT. x 10 FT. Slant Leg Instant U,None,59.98,http://images.acmesports.sports/Quest+Q64+10+F...,4,2,Basketball
1,125,50,957,1,299.98,299.98,50,2013-07-25,5225,CANCELED,50,4,Quest Q64 10 FT. x 10 FT. Slant Leg Instant U,None,59.98,http://images.acmesports.sports/Quest+Q64+10+F...,4,2,Basketball
2,254,112,1014,2,99.96,49.98,112,2013-07-26,5375,CANCELED,112,6,Nike Women's Pro Hyperwarm Half Zip Long Slee,None,31.97,http://images.acmesports.sports/Nike+Women%27s...,6,2,Tennis & Racquet
3,255,112,642,4,120.00,30.00,112,2013-07-26,5375,CANCELED,112,6,Nike Women's Pro Hyperwarm Half Zip Long Slee,None,31.97,http://images.acmesports.sports/Nike+Women%27s...,6,2,Tennis & Racquet
4,256,112,365,2,119.98,59.99,112,2013-07-26,5375,CANCELED,112,6,Nike Women's Pro Hyperwarm Half Zip Long Slee,None,31.97,http://images.acmesports.sports/Nike+Women%27s...,6,2,Tennis & Racquet


In [25]:
product_sales = t4_joined_product_category.groupBy("categoryName").agg(sum("orderItemSubTotal").alias("TotalSales"))

categories_sales = product_sales.orderBy("TotalSales", ascending=False)

categories_sales.limit(5).toPandas()

,categoryName,TotalSales
0,Golf Bags & Carts,1129.75
1,Trade-In,1099.83
2,Golf Shoes,1003.87
3,Tennis & Racquet,979.88
4,NCAA,969.90


In [26]:
categories_sales.write.parquet("categories_sales.parquet")

In [27]:
categories_sales_parquet =spark.read.parquet("categories_sales.parquet")
categories_sales_parquet.limit(5).toPandas()

,categoryName,TotalSales
0,Golf Bags & Carts,1129.75
1,Trade-In,1099.83
2,Golf Shoes,1003.87
3,Tennis & Racquet,979.88
4,NCAA,969.90


## 5. Task

In [45]:
t5_joined = df_order_items.join(df_orders, df_order_items["orderItemOrderId"] == df_orders["orderId"], "inner")
t5_joined.limit(5).toPandas()

,orderItemName,orderItemOrderId,orderItemProductId,orderItemQuantity,orderItemSubTotal,orderItemProductPrice,orderId,orderDate,orderCustomerId,orderStatus
0,1,1,957,1,299.98,299.98,1,2013-07-25,11599,CLOSED
1,2,2,1073,1,199.99,199.99,2,2013-07-25,256,PENDING_PAYMENT
2,3,2,502,5,250.00,50.00,2,2013-07-25,256,PENDING_PAYMENT
3,4,2,403,1,129.99,129.99,2,2013-07-25,256,PENDING_PAYMENT
4,5,4,897,2,49.98,24.99,4,2013-07-25,8827,CLOSED


In [46]:
dates_sales = t5_joined.select("orderItemName", "orderItemSubTotal", "orderDate")
dates_sales.show(5)

+-------------+-----------------+-------------------+
|orderItemName|orderItemSubTotal|          orderDate|
+-------------+-----------------+-------------------+
|            1|           299.98|2013-07-25 00:00:00|
|            2|           199.99|2013-07-25 00:00:00|
|            3|            250.0|2013-07-25 00:00:00|
|            4|           129.99|2013-07-25 00:00:00|
|            5|            49.98|2013-07-25 00:00:00|
+-------------+-----------------+-------------------+
only showing top 5 rows



In [67]:
dates_sales_with_year_month_DESC = dates_sales.groupBy("Year", "Month") \
                                   .agg(sum("orderItemSubTotal") \
                                   .alias("TotalSales")) \
                                   .orderBy(col("TotalSales").desc())

dates_sales_with_year_month_DESC.show(50)

+----+-----+------------------+
|Year|Month|        TotalSales|
+----+-----+------------------+
|2013|   11|3168656.0300002666|
|2013|    9| 2934527.270000178|
|2013|   12|2932964.2700001607|
|2014|    1|2924447.0100001656|
|2014|    3|2862492.2100001364|
|2013|    8| 2828658.700000131|
|2014|    4|2807789.8000001144|
|2014|    2| 2778663.660000113|
|2014|    5|2753078.2200000905|
|2014|    6| 2703463.440000075|
|2013|   10|2624600.6100000516|
|2014|    7|2238496.5199998934|
|2013|    7| 764782.1899999734|
+----+-----+------------------+



In [65]:
get_month = dates_sales_with_year_month_DESC.select("month").first()
get_year = dates_sales_with_year_month_DESC.select("year").first()
get_month = int(get_month[0])
get_year = int(get_year[0])
get_month = get_month - 1
get_year = get_year
get_year, get_month

(2013, 10)

In [66]:
months = ["Ocak", "Şubat", "Mart", "Nisan", "Mayıs", "Haziran", "Temmuz", "Ağustos", "Eylül", "Ekim", "Kasım", "Aralık"]

highest_sales_year_month_in_turkish = months[get_month]

get_year, highest_sales_year_month_in_turkish

(2013, 'Kasım')

## 6. Task

In [78]:
t6_joined = df_order_items.join(df_orders, df_order_items["orderItemOrderId"] == df_orders["orderId"], "inner")

dates_sales = t6_joined.select("orderItemName", "orderItemSubTotal", "orderDate")
dates_sales.show(5)

+-------------+-----------------+-------------------+
|orderItemName|orderItemSubTotal|          orderDate|
+-------------+-----------------+-------------------+
|            1|           299.98|2013-07-25 00:00:00|
|            2|           199.99|2013-07-25 00:00:00|
|            3|            250.0|2013-07-25 00:00:00|
|            4|           129.99|2013-07-25 00:00:00|
|            5|            49.98|2013-07-25 00:00:00|
+-------------+-----------------+-------------------+
only showing top 5 rows



In [91]:
dates_sales_with_day = dates_sales.withColumn("DaysOfWeek", dayofweek("orderDate"))

dates_sales_with_day = dates_sales_with_day.select("orderItemName", "OrderItemSubTotal", "DaysOfWeek")

dates_sales_with_day.show(5)

+-------------+-----------------+----------+
|orderItemName|OrderItemSubTotal|DaysOfWeek|
+-------------+-----------------+----------+
|            1|           299.98|         5|
|            2|           199.99|         5|
|            3|            250.0|         5|
|            4|           129.99|         5|
|            5|            49.98|         5|
+-------------+-----------------+----------+
only showing top 5 rows



In [92]:
dates_sales_with_day_desc = dates_sales_with_day.groupBy("DaysOfWeek").agg(sum("orderItemSubTotal") \
                                                .alias("TotalSales")) \
                                                .orderBy(col("TotalSales").desc())
                                                     

dates_sales_with_day_desc.show(8)

+----------+------------------+
|DaysOfWeek|        TotalSales|
+----------+------------------+
|         6| 5172864.230001591|
|         5| 4977875.180001381|
|         7| 4946465.980001411|
|         3|  4915108.35000137|
|         4| 4904572.310001311|
|         1| 4846988.500001269|
|         2|4558745.3800010085|
+----------+------------------+



In [102]:
get_day = dates_sales_with_day_desc.select("DaysOfWeek").first()
get_day = int(get_day[0])
get_day = get_day
get_day

6

In [104]:
days = ["Pazartesi", "Salı", "Çarşamba", "Perşembe", "Cuma", "Cumartesi", "Pazar"]

highest_sales_day_in_turkish = days[get_day - 1]

highest_sales_day_in_turkish

'Cumartesi'

## 7. Task

In [6]:
#burada da sıfırdan başla.

t7_join = df_order_items.join(df_orders, df_order_items["orderItemOrderId"] == df_orders["orderId"], "inner")

t7_joined_product = t7_join.join(df_products, t7_join["orderItemProductId"] == df_products["productId"], "inner")

t7_joined_product_category = t7_joined_product \
.join(df_categories, t7_joined_product["productCategoryId"] == df_categories["categoryId"], "inner")

t7_joined_product_category_department = \
t7_joined_product_category \
.join(df_departments, t7_joined_product_category["categoryDepartmentId"] == df_departments["departmentId"], "inner")

t7_joined_product_category_department_consumer = \
t7_joined_product_category_department\
.join(df_customers, t7_joined_product_category_department["orderCustomerId"] == df_customers["customerId"], "inner")

t7_joined_product_category_department_consumer.limit(5).toPandas()

,orderItemName,orderItemOrderId,orderItemProductId,orderItemQuantity,orderItemSubTotal,orderItemProductPrice,orderId,orderDate,orderCustomerId,orderStatus,...,departmentName,customerId,customerFName,customerLName,customerEmail,customerPassword,customerStreet,customerCity,customerState,customerZipcode
0,1,1,957,1,299.98,299.98,1,2013-07-25,11599,CLOSED,...,Fan Shop,11599,Mary,Malone,XXXXXXXXX,XXXXXXXXX,8708 Indian Horse Highway,Hickory,NC,28601
1,2,2,1073,1,199.99,199.99,2,2013-07-25,256,PENDING_PAYMENT,...,Fan Shop,256,David,Rodriguez,XXXXXXXXX,XXXXXXXXX,7605 Tawny Horse Falls,Chicago,IL,60625
2,3,2,502,5,250.00,50.00,2,2013-07-25,256,PENDING_PAYMENT,...,Golf,256,David,Rodriguez,XXXXXXXXX,XXXXXXXXX,7605 Tawny Horse Falls,Chicago,IL,60625
3,4,2,403,1,129.99,129.99,2,2013-07-25,256,PENDING_PAYMENT,...,Apparel,256,David,Rodriguez,XXXXXXXXX,XXXXXXXXX,7605 Tawny Horse Falls,Chicago,IL,60625
4,5,4,897,2,49.98,24.99,4,2013-07-25,8827,CLOSED,...,Outdoors,8827,Brian,Wilson,XXXXXXXXX,XXXXXXXXX,8396 High Corners,San Antonio,TX,78240


In [107]:
t7_joined_product_category_department_consumer.printSchema()
"""
CREATE TABLE mytest5 (
    orderItemName integer,
    orderItemOrderId integer,
    orderItemProductId integer,
    orderItemQuantity integer,
    orderItemSubTotal double precision,
    orderItemProductPrice double precision,
    orderId integer,
    orderDate timestamp,
    orderCustomerId integer,
    orderStatus varchar(255),
    productId integer,
    productCategoryId integer,
    productName varchar(255),
    productDescription varchar(255),
    productPrice double precision,
    productImage varchar(255),
    categoryId integer,
    categoryDepartmentId integer,
    categoryName varchar(255),
    departmentId integer,
    departmentName varchar(255),
    customerId integer,
    customerFName varchar(255),
    customerLName varchar(255),
    customerEmail varchar(255),
    customerPassword varchar(255),
    customerStreet varchar(255),
    customerCity varchar(255),
    customerState varchar(255),
    customerZipcode integer
);
"""

root
 |-- orderItemName: integer (nullable = true)
 |-- orderItemOrderId: integer (nullable = true)
 |-- orderItemProductId: integer (nullable = true)
 |-- orderItemQuantity: integer (nullable = true)
 |-- orderItemSubTotal: double (nullable = true)
 |-- orderItemProductPrice: double (nullable = true)
 |-- orderId: integer (nullable = true)
 |-- orderDate: timestamp (nullable = true)
 |-- orderCustomerId: integer (nullable = true)
 |-- orderStatus: string (nullable = true)
 |-- productId: integer (nullable = true)
 |-- productCategoryId: integer (nullable = true)
 |-- productName: string (nullable = true)
 |-- productDescription: string (nullable = true)
 |-- productPrice: double (nullable = true)
 |-- productImage: string (nullable = true)
 |-- categoryId: integer (nullable = true)
 |-- categoryDepartmentId: integer (nullable = true)
 |-- categoryName: string (nullable = true)
 |-- departmentId: integer (nullable = true)
 |-- departmentName: string (nullable = true)
 |-- customerId: i

'\nCREATE TABLE mytest2 (\n    orderItemName integer,\n    orderItemOrderId integer,\n    orderItemProductId integer,\n    orderItemQuantity integer,\n    orderItemSubTotal double precision,\n    orderItemProductPrice double precision,\n    orderId integer,\n    orderDate timestamp,\n    orderCustomerId integer,\n    orderStatus varchar(255),\n    productId integer,\n    productCategoryId integer,\n    productName varchar(255),\n    productDescription varchar(255),\n    productPrice double precision,\n    productImage varchar(255),\n    categoryId integer,\n    categoryDepartmentId integer,\n    categoryName varchar(255),\n    departmentId integer,\n    departmentName varchar(255),\n    customerId integer,\n    customerFName varchar(255),\n    customerLName varchar(255),\n    customerEmail varchar(255),\n    customerPassword varchar(255),\n    customerStreet varchar(255),\n    customerCity varchar(255),\n    customerState varchar(255),\n    customerZipcode integer\n);\n'

## Write to postgres

In [7]:
t7_joined_product_category_department_consumer.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgresql:5432/mydb") \
    .option("dbtable", "mytest4") \
    .option("user", "myuser") \
    .option("password", "mypassword") \
    .option("driver", "org.postgresql.Driver") \
    .mode("overwrite") \
    .save()

In [8]:
t7_df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgresql:5432/mydb") \
    .option("dbtable", "mytest4") \
    .option("user", "myuser") \
    .option("password", "mypassword") \
    .option("driver", "org.postgresql.Driver") \
    .load()

In [9]:
t7_df.limit(5).toPandas()

23/10/11 14:09:56 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,orderItemName,orderItemOrderId,orderItemProductId,orderItemQuantity,orderItemSubTotal,orderItemProductPrice,orderId,orderDate,orderCustomerId,orderStatus,...,departmentName,customerId,customerFName,customerLName,customerEmail,customerPassword,customerStreet,customerCity,customerState,customerZipcode
0,1,1,957,1,299.98,299.98,1,2013-07-25,11599,CLOSED,...,Fan Shop,11599,Mary,Malone,XXXXXXXXX,XXXXXXXXX,8708 Indian Horse Highway,Hickory,NC,28601
1,2,2,1073,1,199.99,199.99,2,2013-07-25,256,PENDING_PAYMENT,...,Fan Shop,256,David,Rodriguez,XXXXXXXXX,XXXXXXXXX,7605 Tawny Horse Falls,Chicago,IL,60625
2,3,2,502,5,250.00,50.00,2,2013-07-25,256,PENDING_PAYMENT,...,Golf,256,David,Rodriguez,XXXXXXXXX,XXXXXXXXX,7605 Tawny Horse Falls,Chicago,IL,60625
3,4,2,403,1,129.99,129.99,2,2013-07-25,256,PENDING_PAYMENT,...,Apparel,256,David,Rodriguez,XXXXXXXXX,XXXXXXXXX,7605 Tawny Horse Falls,Chicago,IL,60625
4,5,4,897,2,49.98,24.99,4,2013-07-25,8827,CLOSED,...,Outdoors,8827,Brian,Wilson,XXXXXXXXX,XXXXXXXXX,8396 High Corners,San Antonio,TX,78240


23/10/11 14:11:48 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /tmp/spark-60f7dd69-1737-48aa-8c3b-ef4aae6c8c33. Falling back to Java IO way
java.io.IOException: Failed to delete: /tmp/spark-60f7dd69-1737-48aa-8c3b-ef4aae6c8c33
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:177)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:113)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:94)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1231)
	at org.apache.spark.util.ShutdownHookManager$.$anonfun$new$4(ShutdownHookManager.scala:65)
	at org.apache.spark.util.ShutdownHookManager$.$anonfun$new$4$adapted(ShutdownHookManager.scala:62)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(Array